# Изучение SQLAlchemy

## Создаем движок

In [2]:
from sqlalchemy import create_engine

## Используется SQLite

In [4]:
engine = create_engine('sqlite:///:memory:')

In [5]:
from sqlalchemy import *

## Определяем таблицу

In [6]:
metadata = MetaData()

In [7]:
vehicles_table = Table('vehicles',
                       metadata,
                       Column('model', String),
                       Column('registration', String),
                       Column('odometer', Integer),
                       Column('last_service', Date),)

In [8]:
vehicles_table

Table('vehicles', MetaData(bind=None), Column('model', String(), table=<vehicles>), Column('registration', String(), table=<vehicles>), Column('odometer', Integer(), table=<vehicles>), Column('last_service', Date(), table=<vehicles>), schema=None)

## Создаем таблицу

In [9]:
vehicles_table.create(bind=engine)

## Генерирование SQL

In [10]:
query = vehicles_table.select()

In [11]:
print(type(query))

<class 'sqlalchemy.sql.selectable.Select'>


In [13]:
print(query)

SELECT vehicles.model, vehicles.registration, vehicles.odometer, vehicles.last_service 
FROM vehicles


In [14]:
query = select([vehicles_table])

In [15]:
print(type(query))

<class 'sqlalchemy.sql.selectable.Select'>


In [16]:
print(query)

SELECT vehicles.model, vehicles.registration, vehicles.odometer, vehicles.last_service 
FROM vehicles


In [18]:
vehicles_table.c.values()

[Column('model', String(), table=<vehicles>),
 Column('registration', String(), table=<vehicles>),
 Column('odometer', Integer(), table=<vehicles>),
 Column('last_service', Date(), table=<vehicles>)]

##Доступ к таблице

In [19]:
vehicles_table.c.odometer

Column('odometer', Integer(), table=<vehicles>)

## Выполнение SQL запроса

In [22]:
query = query.where(vehicles_table.c.odometer < 10000)

In [23]:
print(query)

SELECT vehicles.model, vehicles.registration, vehicles.odometer, vehicles.last_service 
FROM vehicles 
WHERE vehicles.odometer < :odometer_1


In [24]:
results = engine.execute(query)

In [25]:
for row in results:
    print(row)

## Добавим данные

In [26]:
values = [
    {'model': 'Ford Festiva', 'registration': 'HAXOOR', 'odometer': 3141},
    {'model': 'Lotus Elise', 'registration': 'DELEG8', 'odometer': 31415},
]

In [27]:
rows = engine.execute(vehicles_table.insert(), list(values)).rowcount

In [28]:
print(rows, "rows inserted")

2 rows inserted


In [29]:
results = engine.execute(query)

## Запрос

In [30]:
for row in results:
    print(row)

('Ford Festiva', 'HAXOOR', 3141, None)


In [32]:
print(select([vehicles_table.c.model,
              func.sum(vehicles_table.c.odometer).label('total_lm')
             ]).group_by(vehicles_table.c.model))

SELECT vehicles.model, sum(vehicles.odometer) AS total_lm 
FROM vehicles GROUP BY vehicles.model


In [34]:
query = select([vehicles_table.c.model,
                func.sum(vehicles_table.c.odometer).label('total_lm')
               ]).group_by(vehicles_table.c.model)

In [35]:
results = engine.execute(query)

In [36]:
for row in results:
    print(row)

('Ford Festiva', 3141)
('Lotus Elise', 31415)


##ORM

In [53]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

In [54]:
Base = declarative_base()

In [55]:
class Person(Base):
    __tablename__ = 'people' # Choose your own table name!
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    email = Column(String)
    birthday = Column(DateTime)
    appointments = relationship("Appointment", backref="person")

    def __init__(self, firstname, lastname, email):
        """ Constructor is optional """
        self.first_name = firstname
        self.last_name = lastname
        self.email = email

In [56]:
class Appointment(Base):
    __tablename__ = 'appointments'
    id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('people.id')) # <-- Table name
    meeting_at = Column(DateTime)
    notes = Column(String)


In [41]:
print("Underlying table object", repr(Person.__table__))

Underlying table object Table('people', MetaData(bind=None), Column('id', Integer(), table=<people>, primary_key=True, nullable=False), Column('first_name', String(), table=<people>), Column('last_name', String(), table=<people>), Column('email', String(), table=<people>), Column('birthday', DateTime(), table=<people>), schema=None)


In [42]:
print("Mapper that's taking care of things", repr(Person.__mapper__))

Mapper that's taking care of things <Mapper at 0xb1e81b6c; Person>


In [43]:
print("What does the declarative base know?", repr(Base.metadata.tables))

What does the declarative base know? immutabledict({'appointments': Table('appointments', MetaData(bind=None), Column('id', Integer(), table=<appointments>, primary_key=True, nullable=False), Column('person_id', Integer(), ForeignKey('people.id'), table=<appointments>), Column('meeting_at', DateTime(), table=<appointments>), Column('notes', String(), table=<appointments>), schema=None), 'people': Table('people', MetaData(bind=None), Column('id', Integer(), table=<people>, primary_key=True, nullable=False), Column('first_name', String(), table=<people>), Column('last_name', String(), table=<people>), Column('email', String(), table=<people>), Column('birthday', DateTime(), table=<people>), schema=None)})


## Создание таблицы

In [57]:
Base.metadata.create_all(engine) 

## Создание сессии

In [58]:
from sqlalchemy.orm import sessionmaker

In [59]:
Session = sessionmaker(bind=engine)

In [60]:
session = Session()

In [62]:
p = Person('newfirst', 'newlast', 'new@example.com')

In [63]:
session.add(p)

In [65]:
for p in session.query(Person):
    print(p.first_name)

newfirst


In [66]:
print(session.query(Person).filter_by(id=1))

SELECT people.id AS people_id, people.first_name AS people_first_name, people.last_name AS people_last_name, people.email AS people_email, people.birthday AS people_birthday 
FROM people 
WHERE people.id = :id_1


In [67]:
for row in session.query(Person, Person.first_name, Person.email.label('address')).filter_by(id=1):
    print("Person object:", row.Person)
    print("Selected attributes:", row.first_name, row.address)

Person object: <__main__.Person object at 0xb1eab4ac>
Selected attributes: newfirst new@example.com


In [68]:
queried_person = session.query(Person).filter_by(first_name='newfirst').first()

In [69]:
p is queried_person

True

In [70]:
query = session.query(Person).filter(~Person.first_name.in_(['ed', 'wendy', 'jack']))

In [71]:
print(query)

SELECT people.id AS people_id, people.first_name AS people_first_name, people.last_name AS people_last_name, people.email AS people_email, people.birthday AS people_birthday 
FROM people 
WHERE people.first_name NOT IN (:first_name_1, :first_name_2, :first_name_3)


In [72]:
from datetime import datetime, timedelta

In [73]:
query = query.filter(Person.birthday < datetime.now() - timedelta(days=1))

In [74]:
print(query)

SELECT people.id AS people_id, people.first_name AS people_first_name, people.last_name AS people_last_name, people.email AS people_email, people.birthday AS people_birthday 
FROM people 
WHERE people.first_name NOT IN (:first_name_1, :first_name_2, :first_name_3) AND people.birthday < :birthday_1


## Отношения

In [75]:
appointment = Appointment(person=session.query(Person).first(),
                          notes="Appointment date TBC")

In [76]:
session.add(appointment)

In [77]:
for a in session.query(Appointment):
    print(a.person)

In [82]:
for p in session.query(Person):
    print(p.appointments)

# Изучение SQLAlchemy

Основные компоненты SQLAlchemy и их зависимости

![](https://upload.wikimedia.org/wikipedia/commons/4/44/Sqla_arch_small.jpg)

Используем SQLite.

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [4]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [5]:
metadata = MetaData()

In [6]:
users_table = Table('users',
                    metadata,
                    Column('id', Integer, primary_key=True),
                    Column('name', String),
                    Column('fullname', String),
                    Column('password', String),
              )

In [7]:
metadata.create_all(engine)

2015-04-07 16:01:01,658 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2015-04-07 16:01:01,660 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2015-04-07 16:01:01,663 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)




2015-04-07 16:01:01,675 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2015-04-07 16:01:01,677 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [8]:
class User(object):
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password

    def __repr__(self):
        return "<User('%s','%s', '%s')>" % (self.name, self.fullname, self.password)

In [9]:
from sqlalchemy.orm import mapper

In [10]:
mapper(User, users_table)

<Mapper at 0xb1f4490c; User>

In [11]:
user = User("Вася", "Василий", "qweasdzxc")

In [12]:
print(user)

<User('Вася','Василий', 'qweasdzxc')>


In [13]:
print(user.id)

None


In [14]:
from sqlalchemy.ext.declarative import declarative_base

In [15]:
Base = declarative_base()

In [16]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
 
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password
    def __repr__(self):
        return "<User('%s','%s', '%s')>" % (self.name, self.fullname, self.password)

In [17]:
users_table = User.__table__

In [18]:
metadata = Base.metadata

In [19]:
from sqlalchemy.orm import sessionmaker

In [20]:
Session = sessionmaker(bind=engine)

In [21]:
vasiaUser = User("vasia", "Vasiliy Pypkin", "vasia2000")

In [23]:
session = Session()

In [24]:
session.add(vasiaUser)

In [25]:
ourUser = session.query(User).filter_by(name="vasia").first()

2015-04-07 16:01:44,037 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2015-04-07 16:01:44,041 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2015-04-07 16:01:44,043 INFO sqlalchemy.engine.base.Engine ('vasia', 'Vasiliy Pypkin', 'vasia2000')


INFO:sqlalchemy.engine.base.Engine:('vasia', 'Vasiliy Pypkin', 'vasia2000')


2015-04-07 16:01:44,048 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2015-04-07 16:01:44,049 INFO sqlalchemy.engine.base.Engine ('vasia', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('vasia', 1, 0)


In [26]:
session.add_all([User("kolia", "Cool Kolian[S.A.]","kolia$$$"), User("zina", "Zina Korzina", "zk18")])

In [27]:
vasiaUser.password = "-=VP2001=-"

In [28]:
session.dirty

IdentitySet([<User('vasia','Vasiliy Pypkin', '-=VP2001=-')>])

In [29]:
session.new

IdentitySet([<User('zina','Zina Korzina', 'zk18')>, <User('kolia','Cool Kolian[S.A.]', 'kolia$$$')>])

In [30]:
session.commit()

2015-04-07 16:02:28,575 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE users SET password=? WHERE users.id = ?


2015-04-07 16:02:28,581 INFO sqlalchemy.engine.base.Engine ('-=VP2001=-', 1)


INFO:sqlalchemy.engine.base.Engine:('-=VP2001=-', 1)


2015-04-07 16:02:28,586 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2015-04-07 16:02:28,599 INFO sqlalchemy.engine.base.Engine ('zina', 'Zina Korzina', 'zk18')


INFO:sqlalchemy.engine.base.Engine:('zina', 'Zina Korzina', 'zk18')


2015-04-07 16:02:28,604 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [31]:
vasiaUser.id

2015-04-07 16:02:37,007 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2015-04-07 16:02:37,016 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2015-04-07 16:02:37,021 INFO sqlalchemy.engine.base.Engine (1,)


INFO:sqlalchemy.engine.base.Engine:(1,)


1

In [32]:
vasiaUser.name = 'Vaska'

In [33]:
fake_user = User('fakeuser', 'Invalid', '12345')

In [34]:
session.add(fake_user)

In [35]:
session.query(User).filter(User.name.in_(['Vasko', 'fakeuser'])).all()

2015-04-07 16:03:09,959 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE users SET name=? WHERE users.id = ?


2015-04-07 16:03:09,965 INFO sqlalchemy.engine.base.Engine ('Vaska', 1)


INFO:sqlalchemy.engine.base.Engine:('Vaska', 1)


2015-04-07 16:03:09,972 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


...

2015-04-07 16:03:09,987 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2015-04-07 16:03:09,992 INFO sqlalchemy.engine.base.Engine ('Vasko', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('Vasko', 'fakeuser')


[<User('fakeuser','Invalid', '12345')>]

In [36]:
session.rollback()

2015-04-07 16:04:36,182 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


In [37]:
vasiaUser.name

2015-04-07 16:04:43,532 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2015-04-07 16:04:43,541 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2015-04-07 16:04:43,546 INFO sqlalchemy.engine.base.Engine (1,)


INFO:sqlalchemy.engine.base.Engine:(1,)


'vasia'

In [38]:
fake_user in session

False

In [39]:
session.query(User).filter(User.name.in_(['vasia', 'fakeuser'])).all()

2015-04-07 16:05:00,683 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2015-04-07 16:05:00,686 INFO sqlalchemy.engine.base.Engine ('vasia', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('vasia', 'fakeuser')


[<User('vasia','Vasiliy Pypkin', '-=VP2001=-')>]

In [41]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2015-04-07 16:05:42,278 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


2015-04-07 16:05:42,289 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


vasia Vasiliy Pypkin
kolia Cool Kolian[S.A.]
zina Zina Korzina


In [42]:
for name, fullname in session.query(User.name, User.fullname): 
    print(name, fullname)

2015-04-07 16:06:17,345 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


2015-04-07 16:06:17,352 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


vasia Vasiliy Pypkin
kolia Cool Kolian[S.A.]
zina Zina Korzina


In [43]:
for row in session.query(User, User.name).all(): 
    print(row.User, row.name)

2015-04-07 16:06:38,162 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


2015-04-07 16:06:38,167 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User('vasia','Vasiliy Pypkin', '-=VP2001=-')> vasia
<User('kolia','Cool Kolian[S.A.]', 'kolia$$$')> kolia
<User('zina','Zina Korzina', 'zk18')> zina


In [44]:
from sqlalchemy.orm import aliased

In [45]:
user_alias = aliased(User, name='user_alias')

In [47]:
for row in session.query(user_alias, user_alias.name.label('name_label')).all():
    print(row.user_alias, row.name_label)

2015-04-07 16:07:07,656 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password, user_alias.name AS name_label 
FROM users AS user_alias


INFO:sqlalchemy.engine.base.Engine:SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password, user_alias.name AS name_label 
FROM users AS user_alias


2015-04-07 16:07:07,666 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User('vasia','Vasiliy Pypkin', '-=VP2001=-')> vasia
<User('kolia','Cool Kolian[S.A.]', 'kolia$$$')> kolia
<User('zina','Zina Korzina', 'zk18')> zina


In [48]:
for u in session.query(User).order_by(User.id)[1:2]: 
    print(u)

2015-04-07 16:07:26,633 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


2015-04-07 16:07:26,638 INFO sqlalchemy.engine.base.Engine (1, 1)


INFO:sqlalchemy.engine.base.Engine:(1, 1)


<User('kolia','Cool Kolian[S.A.]', 'kolia$$$')>


In [49]:
for name, in session.query(User.name).filter_by(fullname='Vasiliy Pupkin'):
    print(name)

2015-04-07 16:07:47,269 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2015-04-07 16:07:47,276 INFO sqlalchemy.engine.base.Engine ('Vasiliy Pupkin',)


INFO:sqlalchemy.engine.base.Engine:('Vasiliy Pupkin',)


In [50]:
for name, in session.query(User.name).filter(User.fullname=='Vasiliy Pupkin'):
    print(name)

2015-04-07 16:08:55,078 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2015-04-07 16:08:55,086 INFO sqlalchemy.engine.base.Engine ('Vasiliy Pupkin',)


INFO:sqlalchemy.engine.base.Engine:('Vasiliy Pupkin',)


In [51]:
for user in session.query(User).filter(User.name=='vasia').filter(User.fullname=='Vasiliy Pupkin'):
    print(user)

2015-04-07 16:09:10,554 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


2015-04-07 16:09:10,559 INFO sqlalchemy.engine.base.Engine ('vasia', 'Vasiliy Pupkin')


INFO:sqlalchemy.engine.base.Engine:('vasia', 'Vasiliy Pupkin')
